In [29]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

# Чтение данных

In [30]:
data = pd.read_csv('train.csv')
data.sample(n=5)  # посмотрим на 5 случайных строк

,ClientPeriod,MonthlySpending,TotalSpent,Sex,IsSeniorCitizen,HasPartner,HasChild,HasPhoneService,HasMultiplePhoneNumbers,HasInternetService,HasOnlineSecurityService,HasOnlineBackup,HasDeviceProtection,HasTechSupportAccess,HasOnlineTV,HasMovieSubscription,HasContractPhone,IsBillingPaperless,PaymentMethod,Churn
1606,6,29.45,161.45,Female,0,No,No,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,No,Bank transfer (automatic),0
2154,9,103.10,970.45,Male,0,Yes,Yes,Yes,No,Fiber optic,No,No,Yes,Yes,Yes,Yes,Month-to-month,No,Electronic check,0
865,15,96.50,1392.25,Male,1,Yes,No,Yes,Yes,Fiber optic,Yes,No,Yes,No,Yes,No,Month-to-month,Yes,Electronic check,0
262,70,108.15,7930.55,Female,0,Yes,Yes,Yes,Yes,Fiber optic,Yes,Yes,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),0
1062,2,29.85,75.6,Male,0,No,No,No,No phone service,DSL,No,No,No,Yes,No,No,Month-to-month,No,Bank transfer (automatic),1


In [31]:
# название колонок для числовых и категориальных признаков

# числовые признаки
numerical_columns = [
    'ClientPeriod',
    'MonthlySpending',
    'TotalSpent'
]

# категориальные признаки
catigorial_columns = [
    'Sex',
    'IsSeniorCitizen',
    'HasPartner',
    'HasChild',
    'HasPhoneService',
    'HasMultiplePhoneNumbers',
    'HasInternetService',
    'HasOnlineSecurityService',
    'HasOnlineBackup',
    'HasDeviceProtection',
    'HasTechSupportAccess',
    'HasOnlineTV',
    'HasMovieSubscription',
    'HasContractPhone',
    'IsBillingPaperless',
    'PaymentMethod'
]

# все признаки
feature_columns = numerical_columns + catigorial_columns

# целевой признак
target_column = 'Churn'

### Удалим все non'ы

Посмотрим есть ли незаполненные признаки (nan'ы)

In [32]:
data = data.replace('?', np.nan)
data = data.replace('', np.nan)
data = data.replace(' ', np.nan)

In [33]:
data.isna().sum()

ClientPeriod                0
MonthlySpending             0
TotalSpent                  9
Sex                         0
IsSeniorCitizen             0
HasPartner                  0
HasChild                    0
HasPhoneService             0
HasMultiplePhoneNumbers     0
HasInternetService          0
HasOnlineSecurityService    0
HasOnlineBackup             0
HasDeviceProtection         0
HasTechSupportAccess        0
HasOnlineTV                 0
HasMovieSubscription        0
HasContractPhone            0
IsBillingPaperless          0
PaymentMethod               0
Churn                       0
dtype: int64

Видно, что есть nun в колонке TotalSpent. Удалим эти строки с nan.

In [34]:
data = data.dropna()
data.isna().sum()

ClientPeriod                0
MonthlySpending             0
TotalSpent                  0
Sex                         0
IsSeniorCitizen             0
HasPartner                  0
HasChild                    0
HasPhoneService             0
HasMultiplePhoneNumbers     0
HasInternetService          0
HasOnlineSecurityService    0
HasOnlineBackup             0
HasDeviceProtection         0
HasTechSupportAccess        0
HasOnlineTV                 0
HasMovieSubscription        0
HasContractPhone            0
IsBillingPaperless          0
PaymentMethod               0
Churn                       0
dtype: int64

# Применение линейных моделей

### Предобработка данных

In [196]:
numeric_data = data[numerical_columns]
categorial_data = data[catigorial_columns]

In [197]:
# нормировка численных признаков

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
numeric_data = pd.DataFrame(scaler.fit_transform(numeric_data), columns=numerical_columns)
numeric_data.head()

,ClientPeriod,MonthlySpending,TotalSpent
0,0.919099,-1.506436,-0.557582
1,1.612060,-1.295997,-0.184763
2,-1.282072,0.362658,-0.976504
3,-0.018437,0.475334,0.122800
4,1.122911,1.666716,1.968909


In [198]:
# one-hot-encoding для категориальных признаков
categorial_data_dummy = pd.get_dummies(categorial_data)
categorial_data_dummy.head()

,IsSeniorCitizen,Sex_Female,Sex_Male,HasPartner_No,HasPartner_Yes,HasChild_No,HasChild_Yes,HasPhoneService_No,HasPhoneService_Yes,HasMultiplePhoneNumbers_No,...,HasMovieSubscription_Yes,HasContractPhone_Month-to-month,HasContractPhone_One year,HasContractPhone_Two year,IsBillingPaperless_No,IsBillingPaperless_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,False,True,False,True,False,True,False,True,True,...,False,False,True,False,True,False,False,False,False,True
1,0,False,True,False,True,True,False,False,True,False,...,False,False,False,True,True,False,False,True,False,False
2,0,False,True,True,False,True,False,False,True,True,...,False,True,False,False,False,True,False,False,True,False
3,1,True,False,False,True,True,False,False,True,False,...,False,True,False,False,True,False,False,False,False,True
4,0,True,False,False,True,False,True,False,True,False,...,True,False,False,True,True,False,False,True,False,False


In [208]:
datas = [numeric_data, categorial_data_dummy, data[target_column]]
processed_data = pd.concat(datas, axis=1).reindex(categorial_data_dummy.index)
processed_data = processed_data.dropna()
processed_data.head()

,ClientPeriod,MonthlySpending,TotalSpent,IsSeniorCitizen,Sex_Female,Sex_Male,HasPartner_No,HasPartner_Yes,HasChild_No,HasChild_Yes,...,HasContractPhone_Month-to-month,HasContractPhone_One year,HasContractPhone_Two year,IsBillingPaperless_No,IsBillingPaperless_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn
0,0.919099,-1.506436,-0.557582,0.0,False,True,False,True,False,True,...,False,True,False,True,False,False,False,False,True,0.0
1,1.612060,-1.295997,-0.184763,0.0,False,True,False,True,True,False,...,False,False,True,True,False,False,True,False,False,0.0
2,-1.282072,0.362658,-0.976504,0.0,False,True,True,False,True,False,...,True,False,False,False,True,False,False,True,False,1.0
3,-0.018437,0.475334,0.122800,1.0,True,False,False,True,True,False,...,True,False,False,True,False,False,False,False,True,0.0
4,1.122911,1.666716,1.968909,0.0,True,False,False,True,False,True,...,False,False,True,True,False,False,True,False,False,0.0


In [213]:
X = processed_data.drop(columns=target_column)
X

,ClientPeriod,MonthlySpending,TotalSpent,IsSeniorCitizen,Sex_Female,Sex_Male,HasPartner_No,HasPartner_Yes,HasChild_No,HasChild_Yes,...,HasMovieSubscription_Yes,HasContractPhone_Month-to-month,HasContractPhone_One year,HasContractPhone_Two year,IsBillingPaperless_No,IsBillingPaperless_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0.919099,-1.506436,-0.557582,0.0,False,True,False,True,False,True,...,False,False,True,False,True,False,False,False,False,True
1,1.612060,-1.295997,-0.184763,0.0,False,True,False,True,True,False,...,False,False,False,True,True,False,False,True,False,False
2,-1.282072,0.362658,-0.976504,0.0,False,True,True,False,True,False,...,False,True,False,False,False,True,False,False,True,False
3,-0.018437,0.475334,0.122800,1.0,True,False,False,True,True,False,...,False,True,False,False,True,False,False,False,False,True
4,1.122911,1.666716,1.968909,0.0,True,False,False,True,False,True,...,True,False,False,True,True,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5268,-1.200547,-1.145210,-0.973749,0.0,True,False,False,True,False,True,...,True,False,False,True,True,False,False,True,False,False
5269,0.715287,-0.679594,-0.045372,0.0,True,False,False,True,False,True,...,False,True,False,False,False,True,False,False,False,True
5270,-1.282072,-0.328310,-0.985693,0.0,False,True,True,False,True,False,...,True,True,False,False,False,True,False,False,True,False
5271,-0.140724,0.365972,-0.033560,0.0,False,True,True,False,True,False,...,True,False,True,False,False,True,True,False,False,False


In [214]:
y = processed_data[target_column]
y

0       0.0
1       0.0
2       1.0
3       0.0
4       0.0
       ... 
5268    0.0
5269    0.0
5270    1.0
5271    0.0
5272    0.0
Name: Churn, Length: 5264, dtype: float64

### Заголовок

2) С помощью кроссвалидации или разделения на train/valid выборку протестируйте разные значения гиперпараметра C и выберите лучший (можно тестировать С=100, 10, 1, 0.1, 0.01, 0.001) по метрике ROC-AUC.

Если вы разделяете на train/valid, то используйте LogisticRegressionCV. Он сам при вызове .fit() подберет параметр С. (не забудьте передать scroing='roc_auc', чтобы при кроссвалидации сравнивались значения этой метрики, и refit=True, чтобы при потом модель обучилась на всем датасете с лучшим параметром C).


(более сложный вариант) Если вы будете использовать кроссвалидацию, то преобразования данных и LogisticRegression нужно соединить в один Pipeline с помощью make_pipeline, как это делалось во втором семинаре. Потом pipeline надо передать в GridSearchCV. Для one-hot-encoding'a можно испльзовать комбинацию LabelEncoder + OneHotEncoder (сначала превращаем строчки в числа, а потом числа првращаем в one-hot вектора.)

В задании сказано "Для one-hot-encoding'a **можно** испльзовать комбинацию LabelEncoder + OneHotEncoder"

Можно, но не обязательно. Я считаю это излишним, так что использовал только OneHotEncoder.

In [24]:
X = data.drop(columns=target_column)
y = data[target_column]

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer

In [26]:
# трансформер для числовых признаков
numeric_transformer = Pipeline(
    steps=[("scaler", StandardScaler())]
)

# трансформер для категориальных признаков
categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore"))
    ]
)

# препроцессор, который будет обрабатывать и числовые, и категориальные признаки
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numerical_columns),
        ("cat", categorical_transformer, catigorial_columns),
    ]
)

# pipline, который будет сначала делать препроцессинг, потом обучать модель
pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("classifier", LogisticRegression())
    ]
)

pipeline.fit(X, y)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print("model score: %.3f" % pipeline.score(X, y))

model score: 0.806


In [28]:
preprocessor.fit_transform(data).isna().sum()

AttributeError: 'numpy.ndarray' object has no attribute 'isna'